In [5]:
import pandas as pd
from thefuzz import fuzz
import json

# Default columns for the municipality dataframe
default_columns_by_municipality = ["Gemeinde", "Sprachregion", "Kanton", "Bezirk"]

# Metadata columns of interest
columns_metadata = [
    "Title (de)",
    "Date",
    "Institution",
    "Theme 1",
    "Theme 2",
    "Theme 3",
    "Vote trigger",
    "Vote Result status",
    "Official status",
    "Legal act type",
    "Vote trigger actor",
]

# Load data
df_metadata = pd.read_csv("cleaned_metadata_per_votum.csv")
df_municipality = pd.read_csv("cleaned_data_by_municipality.csv")

# Metadata dictionary for storing extracted metadata
valid_columns = []
metadata = {}

counter = 0
counter2 = 0
counter3 = 0

for column in df_municipality.columns:
    if column not in default_columns_by_municipality:
        abstimmung_datum = column[:10]
        abstimmung_name = column[11:]
        metadata_by_date = df_metadata[df_metadata["Date"] == abstimmung_datum]

        # Calculate similarity score
        df_metadata["similarity_score"] = df_metadata["Title (de)"].apply(
            lambda title: fuzz.partial_ratio(abstimmung_name, title)
        )
        metadata_by_name = metadata_by_date[df_metadata["similarity_score"] > 75]
        valid_columns.append(column)
        
        # Check condition
        if metadata_by_name.shape[0] == 1:
            metadata[column] = metadata_by_name.iloc[0]
            counter += 1
        elif metadata_by_name.shape[0] > 1:
            counter2 += 1
        elif metadata_by_name.shape[0] == 0:
            counter3 += 1
print("Equal to 1 ", counter)
print("More than 1 ", counter2)
print("Equal to 0 ", counter3)

# Filter df_municipality to keep only valid columns
columns_to_keep = default_columns_by_municipality + valid_columns
df_municipality_filtered = df_municipality[columns_to_keep]

metadata_df = pd.DataFrame(columns=df_municipality_filtered.columns)
metadata_df = metadata_df[default_columns_by_municipality + valid_columns]

for metadata_type in [
    "Institution",
    "Theme 1",
    "Theme 2",
    "Theme 3",
    "Vote trigger",
    "Vote Result status",
    "Official status",
    "Legal act type",
    "Vote trigger actor",
]:
    

    metadata_row = [
        (
            "unknown"
            if col in default_columns_by_municipality
            else metadata.get(col, {}).get(metadata_type, "unknown")
        )
        for col in df_municipality_filtered.columns
    ]
    
    
    metadata_df.loc[len(metadata_df)] = metadata_row
    metadata_df.iloc[len(metadata_df)-1,0]= metadata_type
   

metadata_df = metadata_df.fillna("unknown")
metadata_df.loc[len(metadata_df)] = [None] * len(metadata_df.columns)

for i, row in df_municipality_filtered.iterrows():
        metadata_df.loc[len(metadata_df)] = row.values

metadata_df.insert(len(default_columns_by_municipality), " ", [None] * len(metadata_df))


empty_col_index = metadata_df.columns[metadata_df.isnull().all()].tolist()
if empty_col_index:
    first_empty_col = empty_col_index[0]
    split_index = metadata_df.columns.get_loc(first_empty_col)

    before_first_empty_col = metadata_df.iloc[:, :split_index]
    after_first_empty_col = metadata_df.iloc[:, split_index + 1 :]
else:
    raise Exception("No empty column found")

first_empty_row_index = metadata_df.isna().all(axis=1).idxmax()
before_first_empty_row = metadata_df.iloc[:first_empty_row_index]
after__first_empty_row = metadata_df.iloc[first_empty_row_index + 1 :]

number_columns_before_first_empty_col = before_first_empty_col.shape[1]
number_rows_before_first_empty_row = before_first_empty_row.shape[0]

raw_data_df = metadata_df.iloc[
    number_rows_before_first_empty_row:, number_columns_before_first_empty_col:
]

dic = {
    "datasetName": "Voting-Data",
    "descriptionText": "This dataset contains voting data",
    "itemNameSingular": "municipality",
    "itemNamePlural": "municipalities",
    "attributeNameSingular": "vote",
    "itemNamePlural": "votes",
    "cellHoverTextSnippet1": "The residents of",
    "cellHoverTextSnippet2": {"single": "have voted", "plural": "have on average voted"},
    "cellHoverTextSnippet3": "'Yes' on",
    "defaultMinValue": raw_data_df.min().min(),
    "defaultMaxValue": raw_data_df.max().max(),
    "defaultColorBreakpoints": "",
    "csvData": metadata_df.to_csv(index=False),
}

with open ("Voting-Data.json", "w") as f:
    f.write(json.dumps(dic, indent=4, ))






/var/folders/w7/3sh8mbdx6p30vgwkwwz5p0vh0000gp/T/ipykernel_30786/1692803448.py:45: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  metadata_by_name = metadata_by_date[df_metadata["similarity_score"] > 75]
/var/folders/w7/3sh8mbdx6p30vgwkwwz5p0vh0000gp/T/ipykernel_30786/1692803448.py:45: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  metadata_by_name = metadata_by_date[df_metadata["similarity_score"] > 75]
/var/folders/w7/3sh8mbdx6p30vgwkwwz5p0vh0000gp/T/ipykernel_30786/1692803448.py:45: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  metadata_by_name = metadata_by_date[df_metadata["similarity_score"] > 75]
/var/folders/w7/3sh8mbdx6p30vgwkwwz5p0vh0000gp/T/ipykernel_30786/1692803448.py:45: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  metadata_by_name = metadata_by_date[df_metadata["similarity_score"] > 75]
/var/folders/w7/3sh8mbdx6p30vgwkwwz5p0vh0000gp/T/ipykernel_3

Equal to 1  426
More than 1  28
Equal to 0  38


/var/folders/w7/3sh8mbdx6p30vgwkwwz5p0vh0000gp/T/ipykernel_30786/1692803448.py:100: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  metadata_df.insert(len(default_columns_by_municipality), " ", [None] * len(metadata_df))
